In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# !pip install hazm

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import hazm
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from tqdm.notebook import tqdm
import os
import re
import json
import copy
import collections

In [4]:
# df=pd.read_csv("/content/drive/MyDrive/Digikala-ProductCategorization/cleaned_dataset.csv")
df=pd.read_csv("/content/drive/MyDrive/Digikala-ProductCategorization/torob_test_cleaned_dataset.csv")
# df=pd.read_csv("torob_test_cleaned_dataset.csv")
df.head(3)

,product_name,cat1_id
0,رژگونه مایع شیگلم رنگ ا,8
1,چای باروتی کلکته گلابی – گرم ا -,2
2,علی تی زنجبیلی علی کافه عددی ا,2


In [5]:
df.head(10)

,product_name,cat1_id
0,رژگونه مایع شیگلم رنگ ا,8
1,چای باروتی کلکته گلابی – گرم ا -,2
2,علی تی زنجبیلی علی کافه عددی ا,2
3,پکیج ایران رادیاتور مدل _ فن دار ا,4
4,تن ماهی گرمی شیلتون,2
5,صندلی تاشو کمپینگ آریامن ا,4
6,اسپرسوساز مباشی مدل ا,4
7,هدفون بلوتوثی شیائومی مدل ا,3
8,کت شلوار مازراتی نخ توییس یقه بلیزری,0
9,گردو درجه یک تویسرکان,2


In [6]:
df =df[df['product_name'].notna()]
print(df.isna().sum(), '\n')

product_name    0
cat1_id         0
dtype: int64 



In [8]:
df['text_len_by_words'] = df['product_name'].apply(lambda t: len(hazm.word_tokenize(t)))
df.head(3)

,product_name,cat1_id,text_len_by_words
0,رژگونه مایع شیگلم رنگ ا,8,5
1,چای باروتی کلکته گلابی – گرم ا -,2,8
2,علی تی زنجبیلی علی کافه عددی ا,2,7


In [9]:
min_max_len = df["text_len_by_words"].min(), df["text_len_by_words"].max()
print(f'Min: {min_max_len[0]} \tMax: {min_max_len[1]}')

Min: 1 	Max: 41


In [10]:
def data_gl_than(data, less_than=100.0, greater_than=0.0, col='text_len_by_words'):
    data_length = data[col].values

    data_glt = sum([1 for length in data_length if greater_than < length <= less_than])

    data_glt_rate = (data_glt / len(data_length)) * 100

    print(f'Texts with word length of greater than {greater_than} and less than {less_than} includes {data_glt_rate:.2f}% of the whole!')

In [11]:
data_gl_than(df, 256, 5)

Texts with word length of greater than 5 and less than 256 includes 69.80% of the whole!


In [ ]:
import plotly.graph_objects as go

fig = go.Figure()

fig.add_trace(go.Histogram(
    x=df['text_len_by_words']
))

fig.update_layout(
    title_text='Distribution of word counts within comments',
    xaxis_title_text='Word Count',
    yaxis_title_text='Frequency',
    bargap=0.2,
    bargroupgap=0.2)

fig.show()

In [13]:
def get_cat_idx(cat,cat_options):
    res=0
    for i,opt in enumerate(cat_options):
        if cat==opt:
            res=i
            break
    return res

## Load Tokenizer

In [14]:
from transformers import BertConfig, BertTokenizer
from transformers import BertModel

from transformers import AdamW
from transformers import get_linear_schedule_with_warmup

import torch
import torch.nn as nn
import torch.nn.functional as F

In [15]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f'device: {device}')

train_on_gpu = torch.cuda.is_available()

if not train_on_gpu:
    print('CUDA is not available.  Training on CPU ...')
else:
    print('CUDA is available!  Training on GPU ...')

device: cuda:0
CUDA is available!  Training on GPU ...


In [16]:
MAX_LEN = 128
TRAIN_BATCH_SIZE = 16
VALID_BATCH_SIZE = 16
TEST_BATCH_SIZE = 16


MODEL_NAME_OR_PATH = 'HooshvareLab/bert-fa-base-uncased'
# OUTPUT_PATH = '/content/bert-fa-base-uncased-sentiment-taaghceh/pytorch_model.bin'
# os.makedirs(os.path.dirname(OUTPUT_PATH), exist_ok=True)

In [17]:
# org_df=pd.read_csv("cleaned_dataset.csv")
org_df=pd.read_csv("/content/drive/MyDrive/Digikala-ProductCategorization/cleaned_dataset.csv")
category1_options=org_df["category1"].unique()


In [18]:
label2id = {label: i for i, label in enumerate(category1_options)}
id2label = {v: k for k, v in label2id.items()}

print(f'label2id: {label2id}')
print(f'id2label: {id2label}')

label2id: {'مد و پوشاک': 0, 'کتاب، لوازم تحریر و هنر': 1, 'کالاهای سوپرمارکتی': 2, 'کالای دیجیتال': 3, 'خانه و آشپزخانه': 4, 'خودرو و موتورسیکلت': 5, 'ابزار آلات و تجهیزات': 6, 'ورزش و سفر': 7, 'زیبایی و سلامت': 8, 'اسباب بازی، کودک و نوزاد': 9, 'محصولات بومی و محلی': 10}
id2label: {0: 'مد و پوشاک', 1: 'کتاب، لوازم تحریر و هنر', 2: 'کالاهای سوپرمارکتی', 3: 'کالای دیجیتال', 4: 'خانه و آشپزخانه', 5: 'خودرو و موتورسیکلت', 6: 'ابزار آلات و تجهیزات', 7: 'ورزش و سفر', 8: 'زیبایی و سلامت', 9: 'اسباب بازی، کودک و نوزاد', 10: 'محصولات بومی و محلی'}


In [ ]:
tokenizer = BertTokenizer.from_pretrained(MODEL_NAME_OR_PATH)
config = BertConfig.from_pretrained(
    MODEL_NAME_OR_PATH, **{
        'label2id': label2id,
        'id2label': id2label,
    })

## Dataset

In [20]:
class DigikalaDataset(torch.utils.data.Dataset):
    """ Create a PyTorch dataset for Digikala. """

    def __init__(self, tokenizer, product_names, targets=None, max_len=128):
        self.product_names = product_names
        self.targets = targets
        self.has_target = isinstance(targets, list) or isinstance(targets, np.ndarray)

        self.tokenizer = tokenizer
        self.max_len = max_len


    def __len__(self):
        return len(self.product_names)

    def __getitem__(self, item):
        product_name = str(self.product_names[item])

        if self.has_target:
            target=self.targets[item]


        encoding = self.tokenizer.encode_plus(
            product_name,
            add_special_tokens=True,
            truncation=True,
            max_length=self.max_len,
            return_token_type_ids=True,
            padding='max_length',
            return_attention_mask=True,
            return_tensors='pt')

        inputs = {
            'product_name': product_name,
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'token_type_ids': encoding['token_type_ids'].flatten(),
        }

        if self.has_target:
            inputs['targets'] = torch.tensor(target, dtype=torch.long)

        return inputs


def create_data_loader(x, y, tokenizer, max_len, batch_size):
    dataset = DigikalaDataset(
        product_names=x,
        targets=y,
        tokenizer=tokenizer,
        max_len=max_len)

    return torch.utils.data.DataLoader(dataset, batch_size=batch_size)

In [21]:
test_data_loader = create_data_loader(df['product_name'].to_numpy(), df['cat1_id'].to_numpy(), tokenizer, max_len=MAX_LEN, batch_size=TEST_BATCH_SIZE)

## Model

In [22]:
class Level1Model(nn.Module):

    def __init__(self, config):
        super(Level1Model, self).__init__()

        self.bert = BertModel.from_pretrained(MODEL_NAME_OR_PATH)
        self.dropout = nn.Dropout(config.hidden_dropout_prob)
        self.classifier = nn.Linear(config.hidden_size, config.num_labels)

    def forward(self, input_ids, attention_mask, token_type_ids):
        bert_output = self.bert(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids)

        pooled_output = self.dropout(bert_output["pooler_output"])
        logits = self.classifier(pooled_output)
        return logits

In [23]:
device

device(type='cuda', index=0)

In [24]:
lvl1_model = Level1Model(config=config)
lvl1_model = lvl1_model.to(device)

print('lvl1_model', type(lvl1_model))

pytorch_model.bin:   0%|          | 0.00/654M [00:00<?, ?B/s]

lvl1_model <class '__main__.Level1Model'>


## Training

In [25]:
EPOCHS = 20
EEVERY_EPOCH = 1000
LEARNING_RATE = 2e-5
CLIP = 0.0
OUTPUT_PATH="/content/drive/MyDrive/Digikala-ProductCategorization/models/"

In [26]:
def simple_accuracy(y_true, y_pred):
    return (y_true == y_pred).mean()

def acc_and_f1(y_true, y_pred, average='weighted'):
    acc = simple_accuracy(y_true, y_pred)
    f1 = f1_score(y_true=y_true, y_pred=y_pred, average=average)
    return {
        "acc": acc,
        "f1": f1,
    }

In [27]:
def y_loss(y_true, y_pred, losses):
    y_true = torch.stack(y_true).cpu().detach().numpy()
    y_pred = torch.stack(y_pred).cpu().detach().numpy()
    y = [y_true, y_pred]
    loss = np.mean(losses)
    return y, loss

In [28]:
def eval_op(model, data_loader, loss_fn):
    model.eval()

    losses = []
    y_pred = []
    y_true = []

    with torch.no_grad():
        for dl in tqdm(data_loader, total=len(data_loader), desc="Evaluation... "):

            input_ids = dl['input_ids']
            attention_mask = dl['attention_mask']
            token_type_ids = dl['token_type_ids']
            targets = dl['targets']


            input_ids = input_ids.to(device)
            attention_mask = attention_mask.to(device)
            token_type_ids = token_type_ids.to(device)
            targets = targets.to(device)

            outputs = model(
                input_ids=input_ids,
                attention_mask=attention_mask,
                token_type_ids=token_type_ids)


            _, preds = torch.max(outputs, dim=1)


            loss = loss_fn(outputs, targets)


            losses.append(loss.item())

            y_pred.extend(preds)
            y_true.extend(targets)

    eval_y, eval_loss = y_loss(y_true, y_pred, losses)
    return eval_y, eval_loss

In [29]:
def train_op(model,
             data_loader,
             loss_fn,
             optimizer,
             scheduler,
             step=0,
             print_every_step=100,
             eval=False,
             eval_cb=None,
             eval_loss_min=np.Inf,
             eval_data_loader=None,
             clip=0.0):

    model.train()

    losses = []
    y_pred = []
    y_true = []

    for dl in tqdm(data_loader, total=len(data_loader), desc="Training... "):
        step += 1
        input_ids = dl['input_ids']
        attention_mask = dl['attention_mask']
        token_type_ids = dl['token_type_ids']
        targets = dl['targets']

        if step<=17000:
          continue

        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)
        token_type_ids = token_type_ids.to(device)
        targets = targets.to(device)

        optimizer.zero_grad()

        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids)

        _, preds = torch.max(outputs, dim=1)


        loss = loss_fn(outputs, targets)

        losses.append(loss.item())

 
        loss.backward()


        if clip > 0.0:
            nn.utils.clip_grad_norm_(model.parameters(), max_norm=clip)


        optimizer.step()


        scheduler.step()

        y_pred.extend(preds)
        y_true.extend(targets)

        if eval:
            train_y, train_loss = y_loss(y_true, y_pred, losses)
            train_score = acc_and_f1(train_y[0], train_y[1], average='weighted')

            if step % print_every_step == 0:
                eval_y, eval_loss = eval_op(model, eval_data_loader, loss_fn)
                eval_score = acc_and_f1(eval_y[0], eval_y[1], average='weighted')

                if hasattr(eval_cb, '__call__'):
                    eval_loss_min = eval_cb(model, optimizer,scheduler,step, train_score, train_loss, eval_score, eval_loss, eval_loss_min)

    train_y, train_loss = y_loss(y_true, y_pred, losses)

    return train_y, train_loss, step, eval_loss_min

In [30]:
def eval_callback(epoch, epochs, output_path):
    def eval_cb(model,optimizer,scheduler, step, train_score, train_loss, eval_score, eval_loss, eval_loss_min):
        statement = ''
        statement += 'Epoch: {}/{}...'.format(epoch, epochs)
        statement += 'Step: {}...'.format(step)

        statement += 'Train Loss: {:.6f}...'.format(train_loss)
        statement += 'Train Acc: {:.3f}...'.format(train_score['acc'])

        statement += 'Valid Loss: {:.6f}...'.format(eval_loss)
        statement += 'Valid Acc: {:.3f}...'.format(eval_score['acc'])

        print(statement)

        if eval_loss <= eval_loss_min:
            print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
                eval_loss_min,
                eval_loss))


            eval_loss_min = eval_loss
            torch.save({'epoch': epoch,'step':step,'model_state_dict': model.state_dict(),'optimizer_state_dict': optimizer.state_dict(),'scheduler_state_dict':scheduler.state_dict(),'eval_loss_min': eval_loss_min,}, output_path+f'checkpoint_ep2_{epoch}.pt')
         

        return eval_loss_min


    return eval_cb

In [31]:
optimizer = AdamW(lvl1_model.parameters(), lr=LEARNING_RATE, correct_bias=False)
total_steps = len(test_data_loader) * EPOCHS
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=0,
    num_training_steps=total_steps
)

loss_fn = nn.CrossEntropyLoss()

step = 0
eval_loss_min = np.Inf
history = collections.defaultdict(list)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:521: FutureWarning:

This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning



In [32]:
last_epoch=1
checkpoint = torch.load(OUTPUT_PATH+f'checkpoint_ep2_1.pt')
lvl1_model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
scheduler.load_state_dict(checkpoint['scheduler_state_dict'])
current_epoch = checkpoint['epoch']
current_epoch=current_epoch+1
step=checkpoint["step"]
eval_loss_min = checkpoint['eval_loss_min']


## Testing

In [36]:
def testing(model, data_loader, tokenizer, max_len=128, batch_size=32):


    predictions = []
    true_targets=[]
    prediction_probs = []


    model.eval()
    with torch.no_grad():
        for dl in tqdm(data_loader, position=0):
            input_ids = dl['input_ids']
            attention_mask = dl['attention_mask']
            token_type_ids = dl['token_type_ids']
            targets = dl['targets']

 
            input_ids = input_ids.to(device)
            attention_mask = attention_mask.to(device)
            token_type_ids = token_type_ids.to(device)
            targets = targets.to(device)



            outputs = model(
                input_ids=input_ids,
                attention_mask=attention_mask,
                token_type_ids=token_type_ids)

            _, preds = torch.max(outputs, dim=1)

            predictions.extend(preds)
            true_targets.extend(targets)
            prediction_probs.extend(F.softmax(outputs, dim=1))


    y_true = torch.stack(true_targets).cpu().detach().numpy()
    y_pred = torch.stack(predictions).cpu().detach().numpy()
    y = [y_true, y_pred]

    return y, prediction_probs

In [37]:
test_y, prediction_probs = testing(lvl1_model, test_data_loader, tokenizer, max_len=128)

  0%|          | 0/126 [00:00<?, ?it/s]

In [38]:
test_score = acc_and_f1(test_y[0], test_y[1], average='weighted')

In [39]:
test_score

{'acc': 0.9279682066567313, 'f1': 0.9424420201474395}